In [38]:
import bisect,heapq,numpy as np
from collections import defaultdict,Counter,OrderedDict,deque
from sortedcontainers import SortedDict
from functools import cmp_to_key
from typing import  *

In [25]:
# Kst starts from 1
def findKstNumber(nums1, nums2, k):
    if len(nums1)>len(nums2):
        nums1,nums2=nums2,nums1
    if not nums1:
        return nums2[k-1]
    if k==1:
        return min(nums1[0],nums2[0])
    kk=k//2
    idx1=kk-1 if kk<=len(nums1) else len(nums1)-1
    idx2=kk-1 if kk<=len(nums2) else len(nums2)-1
    if nums1[idx1]<nums2[idx2]:
        return findKstNumber(nums1[idx1+1:], nums2, k-(idx1+1))
    return findKstNumber(nums1, nums2[idx2+1:], k-(idx2+1))

class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        l1,l2=len(nums1),len(nums2)
        mid=(l1+l2)//2
        if (l1+l2)%2 == 0:
            return (findKstNumber(nums1, nums2, mid)+findKstNumber(nums1, nums2, mid+1))/2
        return findKstNumber(nums1, nums2, mid+1)

In [42]:
def multply(a,b):
    res=0
    while b>0:
        if (b&1)==1: # 奇数，右移要损失1，补上
            res+=a
        b>>=1
        a+=a
    return res
def divide(a,b):
    l,r=0,a # b整数，所以商最大是a
    while l<r:
        mid=(l+r)//2
        v=multply(b,mid)
        if v<=a and v+b>a:
            return mid
        elif v>a:
            r=mid-1
        else:
            l=mid+1
    return l

class Solution:
    def divide(self, dividend: int, divisor: int) -> int:
        a,b=dividend,divisor
        res=divide(abs(a),abs(b))
        if a*b<0:
            res*=-1
        if res>=pow(2, 31):
            return pow(2, 31)-1
        return res

In [13]:
def searchRange1(nums, target):
    start,end=-1,-1
    l,r=0,len(nums)-1
    while l<=r:
        mid=(l+r)//2
        if nums[mid]==target and (mid-1<0 or nums[mid-1]!=target):
            start=mid
            break
        elif nums[mid]>=target:
            r=mid-1
        else:
            l=mid+1
    if start==-1:
        return [-1,-1]
    
    l,r=start,len(nums)-1
    while l<=r:
        mid=(l+r)//2
        v=nums[mid]
        if v==target and (mid+1>=len(nums) or nums[mid+1]!=target):
            end=mid
            break
        elif v<=target:
            l=mid+1
        else:
            r=mid-1
    return [start, end]
    
def searchRange2(nums, target):
    def binarySearchLeft(nums, target): # 找左边界，向左收缩
        l,r=0,len(nums)-1
        while l<r:
            mid=(l+r)//2 # mid靠左，使r=mid时,[1,1]r也向左走
            if nums[mid]==target: # 等于target时不退出，继续搜索
                r=mid    # r可以不变,例如[1,2,3]@2,向左收缩后是[1,2],若-1可能使nums[r]<target。使用靠左mid保证向左收缩。
            elif nums[mid]>target:
                r=mid-1
            else:
                l=mid+1
        return l
    def binarySearchRight(nums, target): # 找右边界，向右收缩
        l,r=0,len(nums)-1
        while l<r:
            mid=(l+r+1)//2 # mid靠右，使l=mid时,[1,1]l也向右走
            if nums[mid]==target: # 等于target时不退出，继续搜索
                l=mid # l可以不变,
            elif nums[mid]<target:
                l=mid+1
            else:
                r=mid-1
        return l
    l,r=binarySearchLeft(nums, target),binarySearchRight(nums, target)
    if nums and nums[l]==target==nums[r]:
        return [l,r]
    return [-1,-1]

def searchRange3(nums, target): # 左闭右开
    def binarySearchLeft(nums, target): # 找左边界，向左收缩
        l,r=0,len(nums)
        while l<r:
            mid=(l+r)//2
            if nums[mid]==target: # 等于target时不退出，继续搜索
                r=mid    # r可以不变,例如[1,2,3]@2,向左收缩后是[1,2],若-1可能使nums[r]<target。使用靠左mid保证向左收缩。
            elif nums[mid]>target:
                r=mid
            else:
                l=mid+1
        return l
    
    def binarySearchRight(nums, target): # 找右边界，向右收缩
        l,r=0,len(nums)
        while l<r:
            mid=(l+r)//2
            if nums[mid]==target: # 等于target时不退出，继续搜索
                l=mid+1
            elif nums[mid]<target:
                l=mid+1
            else:
                r=mid
        return r
    l,r=binarySearchLeft(nums, target),binarySearchRight(nums, target)
    if r<=len(nums) and l<len(nums) and nums[l]==target==nums[r-1]:
        return [l,r-1]
    return [-1,-1]

class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        return searchRange3(nums, target)

In [14]:
Solution().searchRange([5,7,7,8,8,10],8)

[3, 4]

In [71]:
def binarySearch(nums, target):
    l,r=0,len(nums)-1
    while l <= r:
        mid=(l+r)//2
        left,cur,right=nums[l],nums[mid],nums[r]
        if cur==target:
            return mid
        if cur>right: # mid in left
            if target>cur: # in which case target must at right side of mid
                l=mid+1
            else:
                left=binarySearch(nums[:mid], target)
                right=binarySearch(nums[mid+1:], target)
                return left if left is not None else right+mid+1
        else: # mid in right
            if target<cur: # in which case target must at left side of mid
                r=mid-1
            else:
                left=binarySearch(nums[:mid], target)
                right=binarySearch(nums[mid+1:], target)
                return left if left is not None else right+mid+1
    return None

def binarySearch2(nums, target):
    l,r=0,len(nums)-1
    while l <= r:
        mid=(l+r)//2
        left,cur,right=nums[l],nums[mid],nums[r]
        if cur==target:
            return mid
        if cur>right:# mid in left
            if left<=target<mid:
                r=mid-1
            else:
                l=mid+1
        else:
            if cur<target<=right:
                l=mid+1
            else:
                r=mid-1
    return -1

class Solution:
    def search(self, nums: List[int], target: int) -> int:
        # res=binarySearch1(nums, target)
        # return res if res is not None else  -1
        return binarySearch2(nums, target)

In [85]:
def searchInsert2(nums, target):
    l,r=0,len(nums)
    while l < r:
        mid=(l+r)//2
        if nums[mid]==target:
            return mid
        elif nums[mid]<target:
            l=mid+1
        else:
            r=mid
    return r

class Solution:
    def searchInsert(self, nums: List[int], target: int) -> int:
        return searchInsert(nums, target)

In [14]:
def searchMatrix(matrix, target):
    m,n=len(matrix),len(matrix[0])
    l,r=0,m
    while l < r:
        mid=(l+r)//2
        if matrix[mid][0]>=target:
            r=mid
        else:
            l=mid+1
    if l>=m:
        return False
    if matrix[l][0]>target: # not found target in 1st col
        if l==0:
            return False
    while l>0 and matrix[l][0]>target:
        l-=1
        
    row=l
    l,r=0,n
    while l < r:
        mid=(l+r)//2
        if matrix[row][mid]==target:
            return True
        if matrix[row][mid]>target:
            r=mid
        else:
            l=mid+1
    return False
class Solution:
    def searchMatrix(self, matrix: List[List[int]], target: int) -> bool:
        return searchMatrix(matrix, target)

In [25]:
def search(nums, target):
    l,r=0,len(nums)
    while l < r:
        mid=(l+r)//2
        left,cur, right=nums[l],nums[mid],nums[r-1]
        if cur==target:
            return True
        if left==cur==right: # 无法判断，此时首位==cur!=target,可以删去
            l,r=l+1,r-1
        elif cur>right: # 可能在左边
            if left<=target<=cur:
                r=mid-1
            else:
                l=mid+1
        else:
            if cur<=target<=right:
                l=mid+1
            else:
                r=mid-1
    if l<len(nums) and nums[l]==target:
        return True
    return False
        
class Solution:
    def search(self, nums: List[int], target: int) -> bool:
        return search( nums, target)

In [23]:
def findMin1(nums):
    l,r=0,len(nums)
    while l < r:
        mid=(l+r)//2
        left,cur,right=nums[0],nums[mid],nums[-1]
        if left < cur<right:
            r=mid
            continue
        if cur>right: # 左边
            l=mid+1
        else: # 右边
            if mid==0 or nums[mid-1]>cur:
                return cur
            else:
                r=mid
    return nums[l]
        
def findMin2(nums):
    l,r=0,len(nums)
    while l < r:
        mid=(l+r)//2
        left,cur,right=nums[l],nums[mid],nums[r-1]
        if left == cur==right and r-l>1:
            l,r=l+1,r-1
            continue
        if left < cur<right:
            r=mid
            continue
        if cur>right: # 左边
            l=mid+1
        else: # 右边
            if mid==0 or nums[mid-1]>cur:
                return cur
            else:
                r=mid
    return nums[l]

class Solution:
    def findMin(self, nums: List[int]) -> int:
        return findMin2(nums)

In [11]:
def findPeakElement(nums):
    l,r=0,len(nums)
    if r==1:
        return 0
    if nums[0]>nums[1]:
        return 0
    if nums[-2]<nums[-1]:
        return r-1
    
    while l < r:
        mid=(l+r)//2
        v=nums[mid]
        if (mid==len(nums)-1 and nums[mid-1]<v) or ( nums[mid-1]<v and v>nums[mid+1]):
            return mid
        elif v<nums[mid-1]:
            r=mid
        else:
            l=mid+1
    return -1
class Solution:
    def findPeakElement(self, nums: List[int]) -> int:
        return findPeakElement(nums)

In [32]:
def twoSum(numbers,target):
    l,r=0,len(numbers)
    start,end=l,r
    target=target-numbers[start]
    while start<end:
        l,r=start,end
        while l < r:
            mid=(l+r)//2
            v=numbers[mid]
            if v==target:
                return [start+1,mid+1]
            if v>target:
                r=mid
            else:
                l=mid+1
        target+=numbers[start]
        start,end=start+1,l
        target-=numbers[start]
    return [-1,-1]
        
class Solution:
    def twoSum(self, numbers: List[int], target: int) -> List[int]:
        return twoSum(numbers, target)

In [ ]:
def countNodes(root):
    def countLayers(root):
        if not root:
            return 0
        return countLayers(root.left)+1
    layers=countLayers(root)
    if layers==0:
        return 0
    l,r=pow(2,layers-1),pow(2,layers)
    while l<r:
        mid=(l+r)//2
        if exists(root,mid):
            r=mid
        else:
            l=mid+1
    return l

class Solution:
    def countNodes(self, root: TreeNode) -> int:
        return countNodes(root)

In [47]:
def hIndex(citations): #[0,1,5,5,5,6]
    l,r=0,citations[-1]+1
    while l<r:
        mid=(l+r)//2
        low=len(citations)-mid
        if low<0:
            r=mid
            continue
        if citations[low]>=mid:
            l=mid+1
        else:
            r=mid
    return r-1

class Solution:
    def hIndex(self, citations: List[int]) -> int:
        return hIndex(citations)

In [59]:
def lengthOfLIS1(nums):
    n=len(nums)
    dp=[1]*n
    for idx,num in enumerate(nums):
        k=1
        t=0
        for i in range(idx-1,-1,-1):
            if num>nums[i]:
                t=max(t,dp[i])
        dp[idx]=k+t
    return max(dp)
        
        
def lengthOfLIS2(nums):
    n=len(nums)
    stack=[]
    for idx,num in enumerate(nums):
        if not stack:
            stack.append(num)
            continue
        i=bisect.bisect_right(stack, num)
        if num==stack[i-1]:
            continue
        if i>=len(stack):
            stack.append(num)
        else:
            stack[i]=num
    return len(stack)

class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        return lengthOfLIS2(nums)

In [65]:
def findMin(nums):
    l,r=0,len(nums)
    while l < r:
        mid=(l+r)//2
        left,cur,right=nums[l],nums[mid],nums[r-1]
        if r<len(nums):
            right=nums[r]
        if left == cur==right and r-l>1:
            l,r=l+1,r-1
        if left < cur<right:
            r=mid
            continue
        if cur>right: # 左边
            l=mid+1
        elif cur==right: # 左边或右边
            r=r-1
        else: # 右边
            r=mid
            
    return nums[l]

class Solution:
    def findMin(self, nums: List[int]) -> int:
        return findMin(nums)

In [19]:
def merge(n1,n2):
    l,r=0,0
    total=[]
    while l<len(n1) and r<len(n2):
        v1,v2=n1[l],n2[r]
        if v1['val']==v2['val']:
            v1['k']+=v2['k']
            total.append(v1)
            l+=1
        elif v1['val']>v2['val']:
            v1['k']+=(len(n2)-r)
            total.append(v1)
            l+=1
        else:
            total.append(v2)
            r+=1
    if l<len(n1):
        total+=n1[l:]
    if r<len(n2):
        total+=n2[r:]
    return total
            
        

def mergeSort(nums):
    if len(nums) <=1:
        return nums
    mid=len(nums)//2
    l=mergeSort(nums[:mid])
    r=mergeSort(nums[mid:])
    res=merge(l,r)
    return res
    

class Solution:
    def countSmaller(self, nums) :
        nums=[{'val':v,'k':0,'idx':idx}for idx,v in enumerate(nums)]
        total=mergeSort(nums)
        res=[0]*len(nums)
        for i in total:
            idx,k=i['idx'],i['k']
            res[idx]=k
        return res

In [43]:
def partition(p,x):
    low,high =ListNode(),ListNode()
    plow,phigh=low,high
    while p:
        if p.val<x:
            plow.next=p
            plow=p
        else:
            phigh.next=p
            phigh=p
        p=p.next
    phigh.next=None
    plow.next=high.next
    return low.next

class Solution:
    def partition(self, head: ListNode, x: int) -> ListNode:
        return partition(head, x)

In [44]:
t=Solution().partition(newLinkedList([1,4,3,2,5,2]),3)
printList(t)

1 -> 2 -> 2 -> 4 -> 3 -> 5 -> None


In [69]:
def lenList(h):
    if not h:
        return 0
    return 1+lenList(h.next)


def addFunc(num):
    k={'val':num}
    def addon(h,val):
        h.val+=k['val']+val
        if h.val>=10:
            h.val-=10
            k['val']=1
        else:
            k['val']=0
        if k['val']>0 and h.next is None:
            h.next=ListNode(0)
        return k['val']
    return addon

def addTwoNumbers(l1, l2):
    if lenList(l1)<lenList(l2):
        l1,l2=l2,l1
    p1,p2=l1,l2
    addon=addFunc(0)
    while p2:
        addon(p1,p2.val)
        p1,p2=p1.next,p2.next
    while p1 and addon(p1,0)>0:
        p1=p1.next
    return l1

l1,l2=newLinkedList([2,4,3]),newLinkedList([5,6,4])
printList(addTwoNumbers(l1,l2))

7 -> 0 -> 8 -> None


In [76]:
def lenList(h):
    if not h:
        return 0
    return 1+lenList(h.next)

def mergeTwoLists(l1,l2):
    if not l1 or not l1.next:
        if l2:
            l1.next = l2
        return l1
    v1Next,v2 = l1.next.val, l2.val
    if v1Next>v2:
        l2Next=l2.next
        l2.next=l1.next
        l1.next=l2
        mergeTwoLists(l1.next,l2Next)
    else:
        mergeTwoLists(l1.next,l2)
    return l1

class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        list1,list2=newLinkedList(list1),newLinkedList(list2)
        l1,l2=lenList(list1),lenList(list2)
        if l1>l2:
            l1,l2=l2,l1
        if not l1:
            return list2
        return mergeTwoLists(list1, list2)

In [34]:
lenList=lambda l:1+lenList(l.next) if l else 0
KstListNode=lambda x,k: KstListNode(x.next,k-1) if (k!=1 and x) else x

def reverseList(h,prev=None,k=-1): # return new head and the next after orginal last
    if not h: # if h is None, return last element to the head
        return prev,h
    _next=h.next
    h.next=prev
    if k==1:
        return h,_next
    return reverseList(_next,h,k-1)
    
def reverseKGroup(l,k):
    p=h=ListNode(0,l)
    length=lenList(p.next)
    while p and p.next and length>=k:
        cur=p.next
        length-=k
        newHead,_next=reverseList(p.next,None,k)
        p.next.next=_next
        p.next=newHead
        p=cur
    return h.next

class Solution:
    def reverseKGroup(self, head: Optional[ListNode], k: int) -> Optional[ListNode]:
        return reverseKGroup(newLinkedList(head),k)

In [35]:
t=Solution().reverseKGroup([1,2,3,4,5],2)
printList(t)

2 -> 1 -> 4 -> 3 -> 5 -> None


In [33]:

import bisect,heapq,numpy as np
from collections import defaultdict,Counter,OrderedDict,deque
from sortedcontainers import SortedDict
from functools import cmp_to_key
from typing import  *
from queue import PriorityQueue


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
        
    def __len__(self):
        if self.next:
            return 1+len(self.next)
        return 1

    
def newLinkedList(nums):
    if not nums:
        return None
    return ListNode(nums[0],newLinkedList(nums[1:]))

def lenList(h):
    if not h:
        return 0
    return 1+lenList(h.next)

def printList(h):
    if not h:
        print("None")
    else:
        print(f"{h.val} -> ",end='')
        printList(h.next)

def reverseList(h,prev=None,k=-1): # return new head and the next after orginal last
    if not h: # if h is None, return last element to the head
        return prev,h
    _next=h.next
    h.next=prev
    if k==1:
        return h,_next
    return reverseList(_next,h,k-1)

def findKstNode(h,k):
    if not h:
        return None
    if k==1:
        return h
    return findKstNode(h.next,k-1)

h=newLinkedList([1,2,3,4,5])
printList(h)
newH,_next=reverseList(h,None,2)
printList(newH)
printList(_next)
h.next=_next
printList(newH)
printList(findKstNode(h,6))

1 -> 2 -> 3 -> 4 -> 5 -> None
2 -> 1 -> None
3 -> 4 -> 5 -> None
2 -> 1 -> 3 -> 4 -> 5 -> None
None


In [5]:
class TreeNode(object):
    def __init__(self, x,left,right):
        self.val = x
        self.left = left
        self.right = right

def deserialize(vals):
    if not vals:
        return None,vals
    val = vals[0]
    if val == '#':
        return None,vals[1:]
    l,vals = deserialize(vals[1:])
    r,vals = deserialize(vals)
    return TreeNode(val,l,r),vals

class Tree:

    def serialize(self, root):
        if not root:
            return '#'
        l,r=self.serialize(root.left),self.serialize(root.right)
        return f"{root.val} {l} {r}"
        
    def deserialize(self, data):
        vals=data.split(" ")
        root,_=deserialize(vals)
        return root

In [35]:

def rob(root):
    if not roo
    

class Solution:
    def rob(self, root: TreeNode) -> int:
        global not_parent,res
        not_parent,res = 0,0
        rob(root, 0)
        return res

In [36]:
root,_=deserialize([3,2,'#',3,'#','#',3,'#',1,'#','#'])
Solution().rob(root)

6

In [42]:
class Solution:
    def sequenceReconstruction(self, nums: List[int], sequences: List[List[int]]) -> bool:
        from collections import defaultdict
        m=defaultdict(set)
        for s in sequences:
            for idx,num in enumerate(s):
                if idx==0:continue
                m[s[idx-1]].add(num)
        for idx,num in enumerate(nums):
            if idx==len(nums)-1:continue
            _next=nums[idx+1]
            if _next not in m[num]:
                return False
        return True
                


In [43]:
Solution().sequenceReconstruction([1,2,3],[[1,2],[1,3],[1,2,3]])

True

In [50]:
if (a:=20) >18:
    print(a)

20
